<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 作業用ディレクトリへ移動
# システムコマンドは基本「！」をつけるが、ディレクトリ移動は「％」らしい
!mkdir chap6
%cd ./chap6


/content/chap6


In [3]:
# いつものライブラリに加えて、ファインチューニングと性能評価を行うPyTorch Lightningも使用
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# またNo module named ‘torchtext.legacy’エラーが出たため
# https://masaki-note.com/2022/05/27/torchtext_legacy_error/
# に従い以下を実行
# !pip install --upgrade torchtext==0.9.1

# いろいろなライブラリたち
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification


# 日本語事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 33.3 MB/s 
     |████████████████████████████████| 486 kB 73.6 MB/s 
     |████████████████████████████████| 13.4 MB 21.4 MB/s 
     |████████████████████████████████| 582 kB 66.8 MB/s 
     |████████████████████████████████| 880 kB 61.6 MB/s 
     |████████████████████████████████| 120 kB 70.2 MB/s 
     |████████████████████████████████| 6.6 MB 61.8 MB/s 
     |████████████████████████████████| 419 kB 74.4 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=8e93eefbbb1ddc23c4de45de3666085c0fc3a84fae0c6af367daf020988c94b1
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=fff4103af53b8833d47d01289f4c7e90fa925c32b393f00a68e3f67cec2d72d8
  Stored in dire

In [4]:
import pytorch_lightning as pl

In [5]:
# ６章のお題は文章分類
# まずはネガポジ分類からやってみよう
# 入力された文章がポジティブならラベル１、ネガティブならラベル０を返すぞ！

# まずはトークナイザを定義
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# 分類器を定義
# ねがポジ日分類なのでnum_labelはs2
# GPUに乗せておく
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)
bert_sc = bert_sc.cuda()



Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [6]:
''' 
BertForSequenceClassificationには学習と推論の２フェーズがある

推論：符号化した文章を入力して、各カテゴリに対する分類スコアを出力
  分類スコアは２次元配列
  入力のi文目のラベルjに対する分類スコアはscore[i, j]

学習時：符号化した文章とラベル（カテゴリ）を入力として、損失の値を出力


予測性能評価には「精度」という指標を用いる
精度　＝　予測が正しかったデータの数/総データ数 (= 再現率？)

BertForSequenceClassificationはBertModelの最終層の出力のうち、[CLS]に対応する出力に対して、
線形変換、tanh関数、線形変換が適用されて分類スコア計算されている
'''


' \nBertForSequenceClassificationには学習と推論の２フェーズがある\n\n推論：符号化した文章を入力して、各カテゴリに対する分類スコアを出力\n  分類スコアは２次元配列\n  入力のi文目のラベルjに対する分類スコアはscore[i, j]\n\n学習時：符号化した文章とラベル（カテゴリ）を入力として、損失の値を出力\n\n\n予測性能評価には「精度」という指標を用いる\n精度\u3000＝\u3000予測が正しかったデータの数/総データ数 (= 再現率？)\n\nBertForSequenceClassificationはBertModelの最終層の出力のうち、[CLS]に対応する出力に対して、\n線形変換、tanh関数、線形変換が適用されて分類スコア計算されている\n'

In [7]:
# じゃあ実際ネガポジ分類してみようぜ

# 学習用テキスト（レビュー文）
text_list = [
    'この映画は面白かった。',
    'この映画の最後にはガッカリさせられた。',
    'この映画を見て幸せな気持ちになった。'
]

# 1 がポジティブ
label_list = [1, 0, 1]

# データの符号化
encoding = tokenizer(
    text_list,
    padding = 'longest',
    return_tensors='pt'
)


# GPUにのっける
# ちなみに、cuda()はpytorchが提供しているメソッドなので、torch.tensorなどのデータ型にしか使えない
encoding = { k : v.cuda() for k, v in encoding.items()}
labels = torch.tensor(label_list).cuda()
print(type(encoding))
print(type({*encoding}))
print(type({**encoding}))


# 推論フェーズ
# forward は分類スコア（など）を返す。
# 分類スコアは「logits」とタグづけされている。
# 詳しくはhttps://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification.forward
with torch.no_grad():
  output = bert_sc.forward(**encoding)

scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1) # スコア最大のインデックス = カテゴリー番号的なやつ
num_correct = (labels_predicted==labels).sum().item() # 正解数
print(type(labels_predicted))
print(labels_predicted==labels)
print((labels_predicted==labels).sum())

# 精度
accuracy = num_correct / labels.size(0) # 精度
print(labels.size(0))
print(labels.size())


print("# scoresのサイズ：")
print(scores.size())

print("# predicted labels:")
print(labels_predicted)

print("# accuracy:")
print(accuracy)



<class 'dict'>
<class 'set'>
<class 'dict'>
<class 'torch.Tensor'>
tensor([ True, False,  True], device='cuda:0')
tensor(2, device='cuda:0')
3
torch.Size([3])
# scoresのサイズ：
torch.Size([3, 2])
# predicted labels:
tensor([1, 1, 1], device='cuda:0')
# accuracy:
0.6666666666666666


In [8]:
'''
BERTのファインチューニング
符号化した文章を入力して、出力とラベルの間の損失を計算する

損失関数：スコアをSoftmax関数で各ラベルの予測確率に変換し、それと実際のラベルとの間の交差エントロピー
          こいつを最小化したい

損失さえ計算しちゃえば、パラメータ更新作業はPyTorch Lightningが自動でやってくれる

'''

'\nBERTのファインチューニング\n符号化した文章を入力して、出力とラベルの間の損失を計算する\n\n損失関数：スコアをSoftmax関数で各ラベルの予測確率に変換し、それと実際のラベルとの間の交差エントロピー\n          こいつを最小化したい\n\n損失さえ計算しちゃえば、パラメータ更新作業はPyTorch Lightningが自動でやってくれる\n\n'

In [9]:
# さて、損失関数を計算しよう


# 符号化
encoding = tokenizer(
    text_list,
    padding = 'longest',
    return_tensors = 'pt'
)
print(encoding)
print(encoding.items())

# 入力にラベルを追加
# BertForSequenceClassificationは入力にラベル(labels)が含まれる場合に損失を出力
encoding ['labels'] = torch.tensor(label_list)

# GPU送りにする
encoding = { k : v.cuda() for k, v in encoding.items() }


# ロス計算
output = bert_sc(**encoding)
loss = output.loss

print("# loss :")
print(loss)

{'input_ids': tensor([[    2,    70,   450,     9,  9727,   187,    10,     8,     3,     0,
             0,     0,     0,     0,     0,     0],
        [    2,    70,   450,     5,  1133,     7,     9,   444,  3259, 28479,
            26,   191,    84,    10,     8,     3],
        [    2,    70,   450,    11,   212,    16, 13215,    18,  8415,     7,
            58,    10,     8,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}
dict_items([('input_ids', tensor([[    2,    70,   450,     9,  9727,   187,    10,     8,     3,     0,
             0,     0,     0,     0,     0,     0],
        [    2,    70,   450,     5,  1133,   

In [10]:
'''
ここからはlivedoorニュースコーパスを使ってデータ解析してみるぞい

ここに書いてある9個のカテゴリにニュースを分けてみよう
https://www.rondhuit.com/download.html#ldcc
'''

# データダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# ファイル解凍(textフォルダが生成されてそこにデータ解凍される)
!tar -zxf ldcc-20140209.tar.gz

--2022-09-01 21:39:55--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  4.51MB/s    in 1.9s    

2022-09-01 21:39:57 (4.51 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [11]:
# it-life-hack カテゴリの記事を見てみよう
# 一行目：URL
# 二行目：作成日時
# 三行目：記事タイトル
# 四行目以降：本文
# という構成になっているはず

!cat ./text/it-life-hack/it-life-hack-6342280.txt

http://news.livedoor.com/article/detail/6342280/
2012-03-06T13:00:00+0900
USB3.0対応で爆速データ転送！　9倍速のリーダー／ライター登場
USB3.0が登場してから今年で4年目となるがパソコン側でのUSB3.0ポート搭載が進んで来ても対応機器がなかなか充実していない現状がある。そんな中で新しく高速な読み取りが可能なメモリーカードリーダー／ライターが登場した。

バッファローコクヨサプライがUSB3.0対応のカードリーダー／ライターを発表した。SDHC対応のSD系メディアやコンパクトフラッシュ、メモリースティック系メディア、xDピクチャーカードといったデジカメやスマホ、携帯ゲームといった機器で使われている各種メディアを従来よりも短時間でPCに取り込むことが可能になる。

転送速度が5Gbps（理論値）とUSB2.0の480Mbpsと比べて爆速になったUSB3.0はPC側の対応が進んで来ていたが高速転送が生かせる周辺機器としては、外付けHDDや一部のUSBメモリーくらいしかなかった。これに多くのメディアが扱えるリーダー／ライターが加わることで手軽にUSB3.0の恩恵を受けることができるようになる。

今回発表されたのは、USB3.0ケーブルとカードリーダー本体が分かれるタイプの「BSCR09U3」シリーズ（3,240円）、USB3.0コネクタをカードリーダー本体に内蔵している「BSCRD04U3」シリーズ（2,690円）だ。共にホワイトとブラックのカラーバリエーションが用意される（発売は3月下旬以降）。

■リリースページ
■バッファローコクヨサプライ




■バッファローの記事をもっと見る
・約283gでカバンに入る！小型キーボードの驚くべき機能
・3種類のホットキーで使いやすい！AndroidとPCで使えるキーボードの魅力
・ドラえもんもビックリの新アイテム！マウスとキーボードが合体"OPAir"
・ありそうでなかった便利機能！ファイル仕分けする画期的なHDD


サンディスク SanDisk microSDHC 32GB（microSD 32GB） 超高速クラス4  変換アダプター付 世界国内シェアNo.1 バルク品
クチコミを見る


In [12]:
# データローダ
'''
ファインチューニング・性能評価の前に、データセットの前処理を行い、BERTに入力可能な形式に整えることが必要

PyTorchではデータセットをデータローダという形式にする。

ファインチューニングでは、指定された数（バッチサイズ）のデータ（符号化された文章）とラベルをデータセットから抜き出して、ミニバッチと呼ばれるデータ処理の単位を作る

データローダとはデータセットからミニバッチを取り出すもの。

ミニバッチを順番に取り出してはパラメータを更新する
'''


# データローダ生成

# dataset_for_loader: データセットを整形したリスト
# 各要素は単一データを表す。辞書形式の共通のキー(data, labels)を持っている
dataset_for_loader = [
    {'data': torch.tensor([0,1]), 'labels' : torch.tensor(0)},
    {'data': torch.tensor([2,3]), 'labels' : torch.tensor(1)},
    {'data': torch.tensor([4,5]), 'labels' : torch.tensor(2)},
    {'data': torch.tensor([6,7]), 'labels' : torch.tensor(3)},
]

loader = DataLoader(dataset_for_loader, batch_size=2)


# データセットからミニバッチを取り出す
# enumerate : https://note.nkmk.me/python-enumerate-start/
for idx, batch in enumerate(loader):
  print(f'# batch {idx}')
  print(batch)

# ちなみにDataLoaderの定義を以下のようにすると、ランダム順でミニバッチを生成
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)


# データセットからミニバッチを取り出す
# enumerate : https://note.nkmk.me/python-enumerate-start/
for idx, batch in enumerate(loader):
  print(f'# batch {idx}')
  print(batch)

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}
# batch 0
{'data': tensor([[6, 7],
        [0, 1]]), 'labels': tensor([3, 0])}
# batch 1
{'data': tensor([[2, 3],
        [4, 5]]), 'labels': tensor([1, 2])}


In [13]:
'''
実際、前処理を行ってデータセットをBERTに入力可能な形にしてみよう

BERTは一度に512個までしかトークンを受け取れない
今回は１度に１２８トークン送り込むようにする

前処理手順１
各データを次のキーをもつ辞書式にしてデータローダに突っ込む
input_ids
attention_mask
token_type_ids
labels
'''

# カテゴリリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

# トークナイザをロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# データ形式を整える

max_length = 128
# データローダ用に整形したデータセットを入れる用配列
dataset_for_loader = []

# さっきwgetしてきたデータセットは、カテゴリごとに分類されてtextフォルダに入っている
# （text/category名/category名　という階層に記事がある）
# それらをカテゴリごとに分けて、記事を一つ一つ読み込んでは符号化と
# ラベル追加を行い
# dataset_for_loaderに突っ込む

# tqdm : プログレスバーを表示させる（https://vector-ium.com/python-tqdm/）
for label, category in enumerate(tqdm(category_list)):
  # glob : 条件を満たすパスの一覧を再帰的に取得する(https://note.nkmk.me/python-glob-usage/)
  for file in glob.glob(f'./text/{category}/{category}*'):
    # 記事を取得
    # readでファイルの中身を全て読み出し、splintlinesで文字列を改行コードが出てくるごとに分割
    lines = open(file).read().splitlines()

    # ファイルの四行目以降（記事本文だけ）を抜き出す
    # string.join() でstringを間に挟んで配列要素を結合
    text = '\n'.join(lines[3:])

    # 符号化（文字列からトークンID）
    # return_tensors = 'pt' を書いていないので、出力はtorch.tensor型ではないことに注意
    encoding = tokenizer(
        text,
        max_length = max_length,
        padding='max_length',
        truncation=True
    )
    

    # ラベル追加（損失を出力させるため）
    # labelはこの時点ではint型
    encoding['labels'] = label

    # encodingはキーがstring, valueがtorch.tensorのdict型なので、型を変更
    encoding = { k : torch.tensor(v) for k, v in encoding.items() }

    dataset_for_loader.append(encoding)
    




100%|██████████| 9/9 [00:33<00:00,  3.71s/it]


In [14]:
# 前処理が成功したかどうか確認
print(dataset_for_loader[0])

{'input_ids': tensor([    2, 20281,     9,  1037, 28470, 28452,   410,  4812,    12,    71,
        29416,    14,   378,   139,  1419,     8,    36, 26874, 28489,    73,
         1286, 29400,     7,   429, 29575,  1901,    16,  7613,    10,  5602,
            6,  2787,  2935,  2340,   827,   758,   278,  2392,    16,    80,
        26984, 14431, 15060,  1903,     6,  1052, 13363,   130,  6769,  1058,
         2935,  1403,     7,  1037,    29,    11,  9809,    10,  3133,    12,
           28,    80,     6,  3904,  2383,     7,  5457,    16,    33,   687,
           18,  5602,     6,    73,  1286, 29400,     5, 17372,    14, 20908,
        11648,    16,   861, 15743,  4322,     6,  6372, 13690,  2935,  1221,
           12,     6,  7168,     9, 16682,   362,     5,  2013, 14935,    11,
          569,   335,   558,     7,  5342,    16,   546,    10,     8,   192,
          462,  2375,    23,  1344,   573,   465,   811,  1151,    24,    14,
         3904,  1313, 29335, 28457,    16,    33, 

In [35]:
'''
データセットを

学習データ60%
検証データ20%
テストデータ20%

になるようにランダムに分割する

'''

# データセット並べ替え
random.shuffle(dataset_for_loader)

# それぞれのデータ分割のデータ数計算
n = len(dataset_for_loader)
n_train = int(n*0.6)
n_val = int(n*0.2)

# 分割
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:(n_train+n_val)]
dataset_test = dataset_for_loader[(n_train + n_val):]

# データセットから訓練用データローダを作成
# batch_sizeは原論文に合わせて32
# ここでもshuffle=Trueを設定してよりランダムに
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
)

# 検証、テスト用データローダも作成
# こいつらは損失の勾配を計算する必要がないため
# バッチサイズを大きくすることができる
dataloader_val = DataLoader(
    dataset_val, batch_size=256
)

dataloader_test = DataLoader(
    dataset_test, batch_size=256
)

In [16]:
'''
PyTorch Lightningを使って、ファインチューニングとテストを行う

手順
1. 　モデルの振る舞いを記述するクラスを作る
2. ファインチューニングの設定を行う
3. ファインチューニングを行う
'''

'\nPyTorch Lightningを使って、ファインチューニングとテストを行う\n\n手順\n1. \u3000モデルの振る舞いを記述するクラスを作る\n2. ファインチューニングの設定を行う\n3. ファインチューニングを行う\n'

In [29]:
'''
自力で書いた関数
最後に気づいたことによると、根こそぎタブ数が足りないので、動かなかった
模範解答をしたにコピペ

# モデルの振る舞いを記述するクラスを作る
# Pytorch Lightningは、GPUにデータを載せる作業を自動でやってくれるので
# cudaとか書かなくていい

# LightningModuleを継承
class BertForSequenceClassification_pl(pl.LightningModule):

  def __init__(self, model_name, num_labels, lr) :
    # model_name : Transformersのモデル名
    # num_labels : ラベルの数
    # lr : 学習率

    super().__init__()

    # 引数のnum_labelsとlrを保存。
    # 例えば、self.hparams.lrでlrにアクセスできる。
    # チェックポイント作成時にも自動で保存される。
    self.save_hyperparameters() 


    # BERTをロードする
    bert_sc = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

  # 学習データのミニバッチが与えられた時に損失を出力する関数を書く
  # batch_idxはミニバッチの番号だが今回は使わない
  # Pytorch Lightningはこの関数から出力される損失をもとに、パラメータを自動更新する
  def training_step(self, batch, batch_idx):
    output = self.bert_sc(**batch)
    loss = output.loss

    # 損失を'train_loss'の名前でログを取る
    self.log('train_loss', loss)
    return loss

  # 検証用ミニバッチが与えられた時に
  # 検証データを評価する指標（ここではクロスエントロピー（損失関数）とする）を計算する関数を書く
  def validation_step(self, batch, batch_idx):
    output = self.bert_sc(**batch)
    val_loss = output.loss
    # 損失を'val_loss'の名前でログを取る
    self.log('val_loss', loss)

  # テスト用ミニバッチが与えられた時に
  # テストデータを評価する指標を計算する関数を書く
  def test_step(self, batch, batch_idx):
    labels = batch.pop('labels') # ミニバッチからlabelsをポップ（取得）

    output = self.bert_sc(**batch)
    # 予測されたラベルを取得
    labels_predicted = output.logits.argmax(-1)

    # 正解数
    num_correct = (labels_predicted == labels).sum().item()
    # 精度
    accuracy = num_correct / labels.size(0)
    # size : https://pytorch.org/docs/stable/generated/torch.Tensor.size.html#torch.Tensor.size
    print(labels.size(0))
    print(labels.size())

    self.log('accuracy', accuracy)


  # 学習に用いるオプティマイザを返す関数を書く
  def configure_optimizers(self):
    print(type(self.parameters()))
    return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
    '''

In [32]:
# 6-14
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [33]:
# ファインチューニングの設定を行う

# 学習時にモデルの重みを保持する条件を指定
# つまり、いろいろパラメータをいじっていく中で、どんなパラメータであれば保存するのか
checkpoint = pl.callbacks.ModelCheckpoint(
    # val_lossを監視する
    monitor='val_loss',

    # val_lossが最小のモデルを保存
    mode = 'min',

    # val_lossが最も小さいベストモデル１件を保存
    save_top_k=1,

    # モデルの重みのみを保存する
    save_weights_only=True,

    # モデルファイルを保存するディレクトリ
    dirpath='model/'

)


# 学習方法を指定
trainer = pl.Trainer(
    # 学習にGPUを１つ使う
    gpus=1,

    # 学習を10エポック行う
    max_epochs=10,

    # どんな時にモデルの重みを保存するのか指定
    callbacks = [checkpoint]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [34]:
# では、実際ファインチューニングをやろうではないか

# PyTorch Lightning モデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

# ファインチューニングを行う
trainer.fit(model, dataloader_train, dataloader_val)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [36]:
# 学習完了！
# ベストモデルを保存
best_model_path = checkpoint.best_model_path
print('ベストモデルのファイル：', best_model_path)
print('ベストモデルの検証データに対する損失', checkpoint.best_model_score)

ベストモデルのファイル： /content/chap6/model/epoch=7-step=1112.ckpt
ベストモデルの検証データに対する損失 tensor(0.3821, device='cuda:0')


In [37]:
# 学習時の学習データや検証データに対する損失の時間変化を見るには以下
%load_ext tensorboard
%tensorboard --logdir

# 検証データがU字型 -> オーバーフィット気味

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-

In [39]:
# ファインチューニングで得たモデルをテストデータで評価
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]: .2f}')

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1445: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/chap6/model/epoch=7-step=1112.ckpt
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/chap6/model/epoch=7-step=1112.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.9531885981559753
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy:  0.95


In [40]:
# 6-20
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 

# Transformers対応のモデルを./model_transformesに保存
# これは直接読み込める形で保存される
model.bert_sc.save_pretrained('./model_transformers') 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [41]:
# 学習結果を直接読み込むとは
# 以下のように、いつもモデル名を書いていたところにパスを埋め込むことで達成
# 6-21
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers'
)